# Подготовка данных для нейронной сети

Используется соотношение открытого интереса по Call и Put опционам.  
День недели.  
Label в виде 'up', 'down'.  
Запись результатов в файл `nn_features_and_target.csv`  

In [1]:
from pathlib import Path
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import csv

In [2]:
tiker: str = 'RTS'
db_path: Path = Path(fr'c:\Users\Alkor\gd\data_quote_db\{tiker}_futures_options_day_pj1.db')

# Подключение к базе данных
conn = sqlite3.connect(db_path)

# Чтение данных из БД в DataFrame
df_f = pd.read_sql_query("SELECT `TRADEDATE`, `OPEN`, `LOW`, `HIGH`, `CLOSE`, `OPENPOSITION`, `SHORTNAME`, `LSTTRADE` FROM Futures", conn)
df_o = pd.read_sql_query("SELECT `TRADEDATE`, `OPENPOSITION`, `NAME`, `LSTTRADE`, `OPTIONTYPE`, `STRIKE` FROM Options", conn)

# Закрытие соединения
conn.close()

df_f[['TRADEDATE', 'LSTTRADE']] = df_f[['TRADEDATE', 'LSTTRADE']].apply(pd.to_datetime)
df_o[['TRADEDATE', 'LSTTRADE']] = df_o[['TRADEDATE', 'LSTTRADE']].apply(pd.to_datetime)

df_o = df_o[df_o.TRADEDATE < df_o.LSTTRADE]
# df_f['up_down'] = df_f[['OPEN', 'CLOSE']].apply(lambda x: 1 if (x.CLOSE > x.OPEN) else 0, axis=1)  # Добавление колонки направления свечи
df_f['up_down'] = df_f[['OPEN', 'CLOSE']].apply(lambda x: 'up' if (x.CLOSE > x.OPEN) else 'down', axis=1)  # Добавление колонки направления свечи
df_f['next_ud'] = df_f.up_down.shift(-1)  # Добавление колонки с направление следующей свечи
df_f['next_name'] = df_f.SHORTNAME.shift(-1)  # Добавление колонки с именем следующей свечи

# Фильтрация строк, где значения в столбцах SHORTNAME и next_name равны (исключение переходов контракта)
df_f = df_f[df_f['SHORTNAME'] == df_f['next_name']]

# print(df_f.to_string(max_rows=4, max_cols=10))
df_f = df_f.dropna()
# df_f[['next_ud']] = df_f[['next_ud']].astype(int)

# Сортировка по полю даты и сброс индекса
df_f = df_f.sort_values(by='TRADEDATE').reset_index(drop=True)
df_o = df_o.sort_values(by='TRADEDATE').reset_index(drop=True)

# Вывод DataFrame
# print(df_f)
# print(df_o)
print(df_f.to_string(max_rows=4, max_cols=10))
print(df_f.shape)
print('\n', df_o.to_string(max_rows=4, max_cols=10))
print(df_o.shape)

      TRADEDATE      OPEN       LOW      HIGH     CLOSE  ...  SHORTNAME   LSTTRADE up_down next_ud next_name
0    2015-01-05   76930.0   72470.0   78980.0   74600.0  ...   RTS-3.15 2015-03-16    down    down  RTS-3.15
1    2015-01-06   74470.0   71200.0   74610.0   73480.0  ...   RTS-3.15 2015-03-16    down      up  RTS-3.15
...         ...       ...       ...       ...       ...  ...        ...        ...     ...     ...       ...
2362 2024-07-31  109360.0  108280.0  110840.0  110340.0  ...   RTS-9.24 2024-09-19      up    down  RTS-9.24
2363 2024-08-01  110480.0  109950.0  111310.0  110420.0  ...   RTS-9.24 2024-09-19    down    down  RTS-9.24
(2364, 11)

         TRADEDATE  OPENPOSITION      NAME   LSTTRADE OPTIONTYPE  STRIKE
0      2015-01-05         18004  RTS-3.15 2015-01-15          C  100000
1      2015-01-05             0  RTS-3.15 2015-03-16          C   45000
...           ...           ...       ...        ...        ...     ...
765136 2024-08-02             0  RTS-9.24 202

In [3]:
step_strike = 2500

df_rez = pd.DataFrame()

for row in df_f.itertuples():  # Перебираем даты для запроса торгуемых опционов на эту дату
    df = df_o[df_o.TRADEDATE == row.TRADEDATE]
    # print(df.to_string(max_rows=4, max_cols=10))
    df_p = (
        df.query('OPTIONTYPE ==  "P"')
        .groupby(['STRIKE'], as_index=False)
        .agg({'OPENPOSITION': 'sum'})
        .sort_values(['STRIKE'], ascending=True)
        .rename(columns={'OPENPOSITION': 'oi_p'})
    )
    # print(df_p.to_string(max_rows=4, max_cols=10))
    # print(df_p.shape)
    
    df_c = (
        df.query('OPTIONTYPE ==  "C"')
        .groupby(['STRIKE'], as_index=False)
        .agg({'OPENPOSITION': 'sum'})
        .sort_values(['STRIKE'], ascending=True)
        .rename(columns={'OPENPOSITION': 'oi_c'})
    )
    # print(df_c.to_string(max_rows=4, max_cols=10))
    # print(df_c.shape)
    
    # Временный DF для слияния, чтобы исключить пропуски страйков
    df_tmp = pd.DataFrame(columns=['STRIKE'])
    for st in range(df.STRIKE.min(), df.STRIKE.max() + step_strike, step_strike):
        # Новое значение для добавления
        new_row = pd.DataFrame({'STRIKE': [st]})
        # Добавление новой строки в DataFrame с помощью concat
        df_tmp = pd.concat([df_tmp, new_row], ignore_index=True)
    # print(df_tmp.to_string(max_rows=4, max_cols=10))
    
    # Выполнение полного внешнего соединения по столбцу STRIKE
    merged_df = pd.merge(df_p, df_c, on='STRIKE', how='outer')
    merged_df = pd.merge(merged_df, df_tmp, on='STRIKE', how='outer')
    # Приведение типов с помощью infer_objects
    merged_df = merged_df.infer_objects(copy=False)
    
    merged_df = merged_df.fillna(0)
    merged_df[['STRIKE', 'oi_c', 'oi_p']] = merged_df[['STRIKE', 'oi_c', 'oi_p']].astype(int)
    # print(merged_df.to_string(max_rows=4, max_cols=10))
    # print(merged_df.shape)
    
    merged_df['oi_c'] = merged_df['oi_c'].cumsum()
    merged_df['oi_p'] = merged_df.iloc[::-1]['oi_p'].cumsum()[::-1]
    # print(merged_df.to_string(max_rows=4, max_cols=10))
    
    # Записываем в переменную значение цены CLOSE фьючерса
    price_close = df_f.loc[df_f['TRADEDATE'] == row.TRADEDATE, 'CLOSE'].values[0]
    # Расчитываем ближайший страйк
    nearest_strike = round(price_close / step_strike) * step_strike
    # print(price_close, nearest_strike)
    
    # Список индексов со значением nearest_strike из поля merged_df['STRIKE']
    index = merged_df.index[merged_df['STRIKE'] == nearest_strike].tolist()
    # Известный индекс строки (берем 0)
    index = index[0]
    # Определение диапазона строк
    start_index = max(0, index - 10)
    end_index = min(len(df), index + 10 + 1)
    # Получение 10 строк до и 10 строк после строки с известным индексом
    subset_df = merged_df.iloc[start_index:end_index]
    subset_df = subset_df.copy()
    # Создание колонки соотношений отрытого интереса Call и Put
    subset_df['oi'] = subset_df.apply(lambda x: x.oi_p - x.oi_c if x.STRIKE < price_close else x.oi_c - x.oi_p, axis=1)
    # print(subset_df.to_string(max_rows=40, max_cols=10))
    # Извлекаем колонку 'oi' из subset_df
    column_oi_arr = subset_df['oi'].values
    # Нормализация значений в диапазон [0, 1]
    arr_min = np.min(column_oi_arr)
    arr_max = np.max(column_oi_arr)
    normalized_arr = (column_oi_arr - arr_min) / (arr_max - arr_min)
    # Добавляем день недели
    # print(row.TRADEDATE.weekday())
    if row.TRADEDATE.weekday() == 0:
        normalized_arr = np.append(normalized_arr, np.array([1, 0, 0, 0, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 1:
        normalized_arr = np.append(normalized_arr, np.array([0, 1, 0, 0, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 2:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 1, 0, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 3:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 1, 0, 0, 0]))
    elif row.TRADEDATE.weekday() == 4:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 0, 1, 0, 0]))
    elif row.TRADEDATE.weekday() == 5:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 0, 0, 1, 0]))
    elif row.TRADEDATE.weekday() == 6:
        normalized_arr = np.append(normalized_arr, np.array([0, 0, 0, 0, 0, 0, 1]))
    # Добавляем target (label), следующий день повышение или падение
    column_as_row = np.append(normalized_arr, row.next_ud)
    # print(column_as_row)
    # print(type(column_as_row))
    # break
    # Добавляем колонку как строку в DataFrame df_rez
    # Для этого нам нужно транспонировать DataFrame, добавить строку и снова транспонировать обратно
    df_rez = df_rez.T
    df_rez[len(df_rez.columns)] = column_as_row
    df_rez = df_rez.T
    # print(df_rez)

    # break
print(df_rez.to_string(max_rows=8, max_cols=30))

                      0                   1                   2                   3                   4                   5                   6                    7                    8                    9                     10                   11                   12                   13                   14                  15                  16                  17                  18                  19                  20   21   22   23   24   25   26   27    28
0                    1.0  0.9497652206697209  0.9468633252979013  0.8903106399022038  0.8867461070045356  0.8108169863702316  0.7687623530487039   0.7023272286899199   0.6582504084360612   0.5194621211255698                   0.0  0.07172479978064412   0.1569651201316135  0.23787544699471033   0.3427549726376401  0.3859635092369386  0.5616424270813102   0.618663528659073  0.7104159764192439  0.7216579647888128  0.8760982074512448  1.0  0.0  0.0  0.0  0.0  0.0  0.0  down
1                    1.0  0.9996134790400225  0.94

In [4]:
# Сохранение DF в файл без индекса
df_rez.to_csv(fr'../nn_features_and_target.csv', index=False, sep=';')